In [2]:
# Pytorch Imports
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data
import pandas as pd
import datetime
import time

In [52]:
def get_cleaned_data(ticker, pred_window):
# Get stock data from csv
    stock_data = pd.read_csv('./data/{}_extended.csv'.format(ticker), index_col=0).rename_axis('cal_date').reset_index()
    stock_data = stock_data.set_index('cal_date')
    
# Generate new columns
    stock_data['50ma'] = pd.Series.rolling(stock_data['volume'],50, min_periods=50).mean().round()
    stock_data['label'] = stock_data['adj_close'].shift(periods=-pred_window)
    stock_data['label'] = stock_data['label'] - stock_data['adj_close']
    stock_data['label'] = pd.Series(stock_data['label']/stock_data['adj_close']*100).astype(float)
    stock_data['label'] = pd.cut(stock_data['label'], [np.NINF,-2,2,np.inf], labels= [0, 1, 2])
# Format columns as differences
# NOTE: Date is formatted as the difference in days to keep value ranges lower.
    for column in stock_data:
        if column not in ['date', 'label']:
            stock_data[column] = stock_data[column].pct_change()
            
# Drop all cols that contain NAs
    stock_data.dropna(inplace = True)
    
# Refactor date column to be days since last datatpoint and drop NA's again
    stock_data['date'] = stock_data.index
    stock_data['date'] = stock_data['date'].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()), convert_dtype=True)
    stock_data['date'] = stock_data['date'].diff()/86400
    stock_data['date'] = stock_data['date'].round()
    
    stock_data = stock_data.loc[:, ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', '50ma', 'label']]
    stock_data.drop(labels='close', inplace = True, axis = 1)
    stock_data.dropna(inplace = True)
    return stock_data

    
ticker = 'aapl'
pred_window = 5
stock_data = get_cleaned_data(ticker, pred_window)
stock_data

,date,open,high,low,adj_close,volume,50ma,label
cal_date,,,,,,,,
1998-03-18,1.0,-0.018868,0.009367,0.005025,0.022848,-0.324816,-0.015385,1
1998-03-19,1.0,0.033462,0.000000,0.021538,-0.007091,-0.422166,-0.068874,1
1998-03-20,1.0,-0.006699,-0.002598,-0.021084,-0.014165,0.346232,-0.036981,2
1998-03-23,3.0,-0.028100,-0.023074,-0.053077,-0.009539,0.924736,-0.016700,2
1998-03-24,1.0,0.016577,0.066667,0.066206,0.072047,0.629787,-0.009956,1
1998-03-25,1.0,0.047402,-0.008929,0.004571,-0.030020,-0.426903,-0.006161,1
1998-03-26,1.0,-0.031499,-0.027027,0.002655,-0.022157,-0.476046,-0.020845,2
1998-03-27,1.0,-0.004860,0.011481,-0.002648,0.014387,0.259298,-0.016372,1
1998-03-30,3.0,0.004884,0.006957,0.014410,0.018556,-0.019149,-0.015392,0


# Generate Data and Dataloader

In [53]:
# Get input data and labels
signals = stock_data['label'].values
daily_data = stock_data.drop(['label'], axis=1).values

print(signals.shape)
print(daily_data.shape)

(5289,)
(5289, 7)


In [54]:
def dataloader(daily_data, labels, input_length=7, sequence_length=5, batch_size=10):
    # Get total number of days for which we have data -- only want full batches
    days_per_batch = batch_size * sequence_length
    total_days = (len(daily_data) // days_per_batch) * days_per_batch
    
    # Iterate through daily data, at intervals of batch_size X sequence_length
    for ii in range(0, total_days, days_per_batch):
        
        # Get all days in this batch
        batch_days = daily_data[ii: ii+days_per_batch]
        
        # Create the batch/label tensor of the right shape (seq_len x batch_size x input_features)
        batch = torch.zeros((sequence_length, batch_size, input_length), dtype=torch.float64)
        label_data = []
        
        # Fill out this batch/labels
        for batch_num, jj in enumerate(range(0, len(batch_days), sequence_length)):
            sequence_tensor = torch.tensor(batch_days[jj:jj+sequence_length])
            batch[:, batch_num] = sequence_tensor
            
            # Only want labels for day at the end of sequence
            label_data.append(labels[jj+sequence_length-1])
            
        # Fill out label tensor
        label_tensor = torch.tensor(label_data)
        
        yield batch, label_tensor

# Create test and validation data

In [64]:
# Split data into test and validation sets -- will have testing data be first data
# in dataset
test_prop = 0.2
test_end_idx = int(len(daily_data) * test_prop)

# Create testing data
test_features = daily_data[:test_end_idx]
test_labels = signals[:test_end_idx]

# Create training data
train_features = daily_data[test_end_idx:]
train_labels = signals[test_end_idx:]

In [65]:
# Test out the dataloader
sample_batch, sample_labels = next(iter(dataloader(train_features, train_labels)))
print(sample_batch.shape, sample_labels.shape)

torch.Size([5, 10, 7]) torch.Size([10])


# Model Definition

In [66]:
class StockClassifier(nn.Module):
    
    def __init__(self, input_length = 7,lstm_size = 64, lstm_layers=1, output_size = 3, 
                               drop_prob=0.2):
        super().__init__()
        self.input_length = input_length
        self.output_size = output_size
        self.lstm_size = lstm_size
        self.lstm_layers = lstm_layers
        self.drop_prob = drop_prob
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(input_length, lstm_size, lstm_layers, 
                            dropout=drop_prob, batch_first=False)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(lstm_size, output_size)
      
    
    def forward(self, nn_input, hidden_state):
        '''
            Perform a forward pass through the network
            
            Args:
                nn_input: the batch of input to NN
                hidden_state: The LSTM hidden/cell state tuple
                
            Returns:
                logps: log softmax output
                hidden_state: the updated hidden/cell state tuple
        '''
        # Input -> LSTM
        lstm_out, hidden_state = self.lstm(nn_input, hidden)

        # Stack up LSTM outputs -- this gets the final LSTM output for each sequence in the batch
        lstm_out = lstm_out[-1, :, :]
        
        # LSTM -> Dense Layer
        dense_out = self.dropout(self.fc(lstm_out))
        
        # Apply Log Softmax to dense output -- sum denominator across columns
        logps = F.log_softmax(dense_out, dim=1)
                
        # Return the final output and the hidden state
        return logps, hidden_state
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
              weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden

# Test Model

In [67]:
model = StockClassifier(input_length=7, lstm_size=8, lstm_layers=1, output_size=3, drop_prob=0.1).double()
hidden = model.init_hidden(10)
logps, _ = model.forward(batch, hidden)
print(logps)

C:\Users\jonat\Anaconda3\envs\zipline\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


NameError: name 'batch' is not defined

# Train Model

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure that our model is set to 'double' as our volume value requires Float64
model = StockClassifier(input_length=7, lstm_size=128, lstm_layers=2, output_size=3, drop_prob=0.2).double()
model.to(device)

StockClassifier(
  (lstm): LSTM(7, 128, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=128, out_features=3, bias=True)
)

In [69]:
epochs = 1
batch_size = 2
learning_rate = 0.003
clip = 5
input_length=7

print_every = 10
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting Epoch {}'.format(epoch+1))
    steps = 0
    
    for train_batch, labels in dataloader(train_features, train_labels, batch_size=batch_size, input_length=input_length):
        steps += 1
        
        #print("Train Batch: ", train_batch.shape, "Labels Size: ", labels.shape)
        
        # Initialize Hidden/Cell state -- batch size is dynamic to account for batches that are not full
        hidden = model.init_hidden(train_batch.shape[1])
        hidden = tuple([each.data for each in hidden])
        
        # Set tensors to correct device -- GPU or CPU
        train_batch, train_labels = train_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
            
        # Zero out gradients
        optimizer.zero_grad()
        
        # Run data through model -- output is output and new hidden/cell state
        output, hidden = model(train_batch, hidden)
        
        # Calculate loss and perform back prop -- clip grads if necessary
        loss = criterion(output, train_labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # Take optimizer step
        optimizer.step()
        
        # VALIDATION OF MODEL#
        if steps % print_every == 0:
            model.eval()
            val_losses = []
            accuracy = []
            with torch.no_grad():
                for val_batch, val_labels in dataloader(test_features, test_labels, batch_size=batch_size, input_length=input_length):

                    #Init hidden state -- again we have a dynamic batch size here
                    val_hidden = model.init_hidden(val_batch.shape[1])
                    val_hidden = tuple([each.data for each in val_hidden])

                    # Set device for tensors
                    val_batch, val_labels = val_batch.to(device), val_labels.to(device)
                    for each in val_hidden:
                        each.to(device)

                    # Run data through network
                    val_out, val_hidden = model(val_batch, val_hidden)

                    # Calculate and record loss
                    val_loss = criterion(val_out, val_labels)
                    val_losses.append(val_loss.item())

                    # Calculate accuracy of predictions
                    ps = torch.exp(val_out)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == val_labels.view(*top_class.shape)
                    accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())

            # Print out metrics
            print('Epoch: {}/{}...'.format(epoch+1, epochs),
                  'Step: {}...'.format(steps),
                  'Train Loss: {:.6f}...'.format(loss.item()),
                  'Val Loss: {:.6f}...'.format(np.mean(val_losses)),
                  'Accuracy: {:.6f}%...'.format(np.mean(accuracy) * 100))
            
            # Set back to training mode
            model.train()

Starting Epoch 1
Epoch: 1/1... Step: 10... Train Loss: 0.000257... Val Loss: 4.626501... Accuracy: 50.000000%...
Epoch: 1/1... Step: 20... Train Loss: 0.346839... Val Loss: 6.251774... Accuracy: 50.000000%...
Epoch: 1/1... Step: 30... Train Loss: 0.346626... Val Loss: 6.840321... Accuracy: 50.000000%...
Epoch: 1/1... Step: 40... Train Loss: 0.000073... Val Loss: 7.086337... Accuracy: 50.000000%...
Epoch: 1/1... Step: 50... Train Loss: 0.000000... Val Loss: 7.194113... Accuracy: 50.000000%...
Epoch: 1/1... Step: 60... Train Loss: 0.000058... Val Loss: 7.253803... Accuracy: 50.000000%...
Epoch: 1/1... Step: 70... Train Loss: 0.000078... Val Loss: 7.287695... Accuracy: 50.000000%...
Epoch: 1/1... Step: 80... Train Loss: 0.000000... Val Loss: 7.313162... Accuracy: 50.000000%...
Epoch: 1/1... Step: 90... Train Loss: 0.000127... Val Loss: 7.330127... Accuracy: 50.000000%...
Epoch: 1/1... Step: 100... Train Loss: 0.000000... Val Loss: 7.352118... Accuracy: 50.000000%...
Epoch: 1/1... Step: 11

The training loop will error out if you try to run it multiple times. This happens because the state of the dataloaders has not changed since the last run, and therefore you'll run out of data very quickly. When this happens, go back to the 'Create test and validation data' cell and re-run. This will reset the data in the generators and allow you to try and train again.

## Old Get stock data function

In [4]:
# Function to generate two csv files for training/testing
def get_stock_data(file_name):
    ''' 
        Will divide a dataset into both test and training sets
        
        Args:
        
        file_name (string): path to the csv file containing all data
        percent_test (float): percentage of dataset to set aside for testing
        
        returns nothing
    '''
    # Open file and place into dataframe
    stock_data = pd.read_csv(file_name)
    stock_data['date'] = stock_data['date'].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()), convert_dtype=True)
    stock_data = stock_data.dropna()
    return stock_data

# Get stock data from csv
stock_data = get_stock_data('./data/aapl.csv')
stock_data

,Unnamed: 0,action,close,column_index,date,direction,high,low,move,open,...,scale_value,signal,status,volume,50ma,volume_ratio,window_delta,window_direction,pct_change,label
0,0,2,78.85,8,1.332227e+09,0,78.97,75.73,3,78.01,...,75.96,1.0,bull correction,204165500,138824658.0,1.470672,1.11,1,1.407736,1
1,1,4,78.40,9,1.332313e+09,1,79.33,78.26,4,78.43,...,79.05,1.0,bull confirmed,161010500,140074746.0,1.149461,1.96,1,2.500000,2
2,2,0,77.99,9,1.332400e+09,1,78.66,77.49,0,77.78,...,78.26,1.0,bull confirmed,155967700,141903118.0,1.099114,1.37,1,1.756635,1
3,3,0,77.56,9,1.332486e+09,1,78.31,77.34,0,78.14,...,78.26,1.0,bull confirmed,107622200,142980138.0,0.752707,0.45,1,0.580196,1
4,4,0,78.98,9,1.332745e+09,1,79.00,77.46,0,78.04,...,78.26,1.0,bull confirmed,148935500,144895912.0,1.027879,1.52,1,1.924538,1
5,5,1,79.96,9,1.332832e+09,1,80.19,78.86,1,78.88,...,79.84,1.0,bull confirmed,151782400,146801452.0,1.033930,1.93,1,2.413707,2
6,6,1,80.36,9,1.332918e+09,1,80.86,79.41,1,80.46,...,80.63,1.0,bull confirmed,163865100,148864268.0,1.100769,0.88,1,1.095072,1
7,7,0,79.36,9,1.333004e+09,1,80.23,79.01,0,79.74,...,79.84,1.0,bull confirmed,152059600,150521504.0,1.010218,3.10,1,3.906250,2
8,8,2,78.01,10,1.333091e+09,0,79.45,77.80,3,79.21,...,78.26,1.0,bull correction,182759500,152868002.0,1.195538,4.78,1,6.127420,2
9,9,0,80.50,10,1.333350e+09,0,80.52,78.12,0,78.31,...,78.26,1.0,bull correction,149587900,153789888.0,0.972677,1.27,1,1.577640,1
